In [1]:
from src.dataset.Dataset import Dataset
from os.path import join,basename
from os import makedirs
import json
from tqdm import tqdm
from experiments import DataRefiner
from dataset.DataPreprocessor import DataPreprocessor
from labelregions.LabelRegionLoader import LabelRegionLoader
from experiments.CrossValidationTraining import CrossValidationTraining
import experiments.EdgePropabilityCallback as EdgePropabilityCallback

In [2]:
DATA_DIR = '../data'

In [3]:
DECO = Dataset(join(DATA_DIR, "Deco"), "Deco")
FUSTE = Dataset(join(DATA_DIR, "FusTe"), "FusTe")
TEST = Dataset(join(DATA_DIR, "Test"), "Test")

In [4]:
datasets = dict([(ds.name, ds) for ds in [DECO, FUSTE, TEST]])
dataset = datasets['Deco']
data_preprocessor = DataPreprocessor(DATA_DIR, "preprocessed_annotations_elements.json")
data_preprocessor.preprocess(dataset.name)

In [5]:
DataRefiner.refine(dataset)

In [6]:
label_region_loader = LabelRegionLoader(introduce_noise=False)

In [7]:
edge_probability_callback = EdgePropabilityCallback.default_edge_mutation_probability_callback
experiment_class = CrossValidationTraining

In [8]:
import uuid

k=10
weight_tuning_rounds = 10
search_rounds = 10
noise_part = 'noise' if label_region_loader.introduce_noise else 'no_noise'
run_id = uuid.uuid1().hex
dir_name = f"{noise_part}_{1}_{run_id}"
out_path = join('../jupy/output/', 'NoImprovement', dataset.name, dir_name)
makedirs(out_path, exist_ok=True)

In [9]:
from random import shuffle
from typing import List, Dict
from numpy import array_split

def get_folds() -> List[Dict[str, List]]:
    """Creates folds for cross validation while balancing single and multi table files per fold"""
    single_table_keys = dataset.single_table_keys
    multi_table_keys = dataset.multi_table_keys

    shuffle(single_table_keys)
    shuffle(multi_table_keys)

    single_table_chunks = array_split(single_table_keys, k)
    multi_table_chunks = array_split(multi_table_keys, k)

    # Combine Single and Multi Table Chunks to test folds
    test_chunks = [list(s_chunk) + list(m_chunk) for s_chunk, m_chunk in
                   zip(single_table_chunks, multi_table_chunks)]
    # Train fold is all keys without the test fold
    train_chunks = [list(set(dataset.keys).difference(test_chunk)) for test_chunk in test_chunks]

    return [{"train": train_chunk, "test": test_chunk} for train_chunk, test_chunk in
            zip(train_chunks, test_chunks)]

def dump(file_name, data, subdir=""):
    """Dump given data to a json file"""
    if subdir != "":
        makedirs(join(out_path, subdir), exist_ok=True)
    with open(join(out_path, subdir, file_name), "w") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [10]:
folds = get_folds()
dump("config.json", {
            "dataset": dataset.name,
            "noise": label_region_loader.introduce_noise,
            "k": k,
            "weight_tuning_rounds": weight_tuning_rounds,
            "search_rounds": search_rounds,
            "seed": 1,
        })
dump("folds.json", dict([(i, fold) for i, fold in enumerate(folds)]))


In [11]:
from src.experiments import Analyser
from src.search.GeneticSearchConfiguration import GeneticSearchConfiguration
from src.search.GeneticSearch import GeneticSearch
from src.graph.SpreadSheetGraph import SpreadSheetGraph
from src.search.FitnessRater import FitnessRater, get_initial_weights
from labelregions.BoundingBox import BoundingBox
from scipy.optimize import minimize, Bounds
from graph.Edge import Edge
from typing import Union, Callable
edge_mutation_probability_callback: Callable[[Edge], int] = lambda x: 1
def genetic_search_accuracy(
        ground_truth: List[BoundingBox],
        sheet_graph: SpreadSheetGraph,
        rater: FitnessRater,
):
    """Runs genetic searches, evaluates the results against the ground truth, and returns the avg. accuracy score"""
    search = GeneticSearch(
        sheet_graph,
        rater,
        GeneticSearchConfiguration(
            sheet_graph,
            edge_mutation_probability_callback=edge_mutation_probability_callback,
        ),
    )
    # Genetic Search runs multiple times and gets averaged
    results = [search.run() for _ in range(search_rounds)]

    print('------------------遗传搜索result---------------------')
    for i,result in  enumerate(results):
        print(f' -- result -- {i}:')
        for item in result.nodes:
            print(f'type:{item.type}',end='\t')
            print(f'left:{item.left}',end='\t')
            print(f"top:{item.top}",end='\t')
            print(f'right:{item.right}',end='\t')
            print(f'bottom:{item.bottom}')
    print('---------------------------------------------')

    accuracies = [
        Analyser.accuracy_based_on_jacard_index(ground_truth, result.get_table_definitions())
        for result in results
    ]
    return sum(accuracies) / len(accuracies)

def process_fold( fold: Dict[str, List], fold_num: int) -> float:
    """Evaluates on fold of the cross validation, returns the accuracy of the fold"""
    # Train multiple rounds
    # weights_and_errors = [
    #     train(fold["train"], fold_num, i)
    #     for i in tqdm(range(weight_tuning_rounds), desc=f"Training Rounds of fold {fold_num}")
    # ]
    # Average the training results weighted by their error

    #加载权重
    with open(r'D:\ShareWithMac\output\NoImprovement\Deco\no_noise_1_fe301df8eb2111eda40ee0be0345a1e0\fold_9\fold_9_weights.json') as file:
        weights_and_errors = json.load(file)
        for e in weights_and_errors['weights_and_errors']:
            print(e['error_rate'])
    weights = CrossValidationTraining.weighted_average(weights_and_errors['weights_and_errors'])



    dump(
        f"fold_{fold_num}_weights.json",
        {"weights_and_errors": weights_and_errors, "weights": weights},
        subdir=f"fold_{fold_num}"
    )
    # Test accuracies on gold standard
    # Disable any noise
    label_region_loader.introduce_noise = False
    file_accuracies = {}
    for key in tqdm(fold["test"], desc=f"Test Set Validation of fold {fold_num}"):
        # Get ground truth data
        print('##################################################################################################################################################')
        sheet_data = dataset.get_specific_sheetdata(key, label_region_loader)
        print('------------------Sheetdata---------------------')
        for item in sheet_data.label_regions:
            print(f'type:{item.type}',end='\t')
            print(f'left:{item.left}',end='\t')
            print(f"top:{item.top}",end='\t')
            print(f'right:{item.right}',end='\t')
            print(f'bottom:{item.bottom}')


        print('---------------------------------------------')
        sheet_graph = SpreadSheetGraph(sheet_data)
        ground_truth = sheet_graph.get_table_definitions()

        print('------------------Truth---------------------')
        for item in ground_truth:
            print(item)
        print('---------------------------------------------')

        print('*********************************')
        print(f'sheet_data:{sheet_data}')
        print(f'sheet_graph:{sheet_graph}')
        print(f'ground_truth:{ground_truth}')
        print('*********************************')

        # Evaluate the prediction
        rater = FitnessRater(weights)
        if len(sheet_graph.nodes) <= 10:
            accuracy = CrossValidationTraining.exhaustive_search_accuracy(ground_truth, sheet_graph, rater)
        else:
            accuracy = genetic_search_accuracy(ground_truth, sheet_graph, rater)
        file_accuracies[key] = accuracy
        print('##################################################################################################################################################')
    # Average fold accuracies of test data
    fold_accuracy = sum(file_accuracies.values()) / len(file_accuracies.values())
    dump(
        f"fold_{fold_num}_file_accuracies.json",
        {"fold_file_accuracies": file_accuracies, "fold_accuracy": fold_accuracy},
        subdir=f"fold_{fold_num}",
    )
    return fold_accuracy

def train( train_keys: List[str], fold_num: int, training_round: int) -> Dict[str, Union[List[float], float]]:
    """Performs SQP on the given keys, outputs the resulting weights and their error rate"""
    graphs = [
        SpreadSheetGraph(dataset.get_specific_sheetdata(key, label_region_loader))
        for key in train_keys
    ]
    partitions = {}
    for graph in graphs:
        # Create more alternative partitions on multi table files (10 alternatives per table in file)
        partitions[graph] = CrossValidationTraining.generate_alternatives(graph, 10 * len(graph.get_components()))
    dump(
        f"fold_{fold_num}_training_round_{training_round}_input.json",
        dict([(str(graph.sheet_data), partitions) for graph, partitions in partitions.items()]),
        subdir=join(f"fold_{fold_num}", "training")
    )
    initial_weights = get_initial_weights()
    # Create rater object outside to leverage caching
    rater = FitnessRater(initial_weights)
    # Use SQP to minimize the obj. function
    res = minimize(
        CrossValidationTraining.objective_function,
        initial_weights,
        args=(partitions, rater),
        method="SLSQP",
        bounds=Bounds(0, 1000),
    )
    weights = list(res.x)
    rater.weights = weights
    # Calculate error rate components
    total_alternative_count = 0
    better_than_original_alternative_count = 0
    for graph, alternatives in partitions.items():
        obj_score_original_graph = rater.rate(graph, graph.edge_toggle_list)
        total_alternative_count += len(alternatives)
        better_than_original_alternative_count += len([
            alternative
            for alternative in alternatives
            if rater.rate(graph, alternative) < obj_score_original_graph
        ])
    dump(
        f"fold_{fold_num}_training_round_{training_round}_result.json",
        {
            "weights": weights,
            "total_alternative_count": total_alternative_count,
            "better_than_original_alternative_count": better_than_original_alternative_count,
        },
        subdir=join(f"fold_{fold_num}", "training"),
    )
    return {"weights": weights, "error_rate": better_than_original_alternative_count / total_alternative_count}

In [12]:
fold_accuracies = []
for fold_num_and_fold in tqdm(enumerate(folds)):
    fold_accuracies.append(process_fold(fold_num_and_fold[1], fold_num_and_fold[0]))

0it [00:00, ?it/s]

0.021444444444444443
0.019222222222222224
0.022555555555555554
0.022555555555555554
0.02188888888888889
0.025444444444444443
0.021
0.023555555555555555
0.021444444444444443
0.022444444444444444



Test Set Validation of fold 0:   0%|          | 0/63 [00:00<?, ?it/s]

##################################################################################################################################################
------------------Sheetdata---------------------
type:LabelRegionType.HEADER	left:1	top:4	right:9	bottom:4
type:LabelRegionType.HEADER	left:1	top:6	right:1	bottom:6
type:LabelRegionType.DATA	left:2	top:6	right:7	bottom:6
type:LabelRegionType.DATA	left:2	top:7	right:3	bottom:8
---------------------------------------------
------------------Truth---------------------
"top": 4
"left": 1
"bottom": 8
"right": 1
---------------------------------------------
*********************************
sheet_data:Sheetdata(Mary of ../data\Deco\xls\mary_hain__25909__Jan-Feb Internal Resource.xlsx
sheet_graph:<src.graph.SpreadSheetGraph.SpreadSheetGraph object at 0x000002155605ED30>
ground_truth:[<labelregions.BoundingBox.BoundingBox object at 0x000002155605E0D0>]
*********************************
------------------详尽搜索result---------------------
type:LabelRegi


Test Set Validation of fold 0:   3%|▎         | 2/63 [00:14<07:24,  7.29s/it]

------------------遗传搜索result---------------------
 -- result -- 0:
type:LabelRegionType.HEADER	left:3	top:3	right:3	bottom:3
type:LabelRegionType.HEADER	left:1	top:4	right:4	bottom:4
type:LabelRegionType.DATA	left:1	top:5	right:4	bottom:6
type:LabelRegionType.DATA	left:1	top:7	right:1	bottom:7
type:LabelRegionType.DATA	left:3	top:7	right:4	bottom:7
type:LabelRegionType.DATA	left:1	top:9	right:4	bottom:10
type:LabelRegionType.DATA	left:1	top:11	right:1	bottom:11
type:LabelRegionType.DATA	left:3	top:11	right:4	bottom:11
type:LabelRegionType.DATA	left:1	top:13	right:4	bottom:14
type:LabelRegionType.DATA	left:1	top:15	right:1	bottom:15
type:LabelRegionType.DATA	left:3	top:15	right:4	bottom:15
 -- result -- 1:
type:LabelRegionType.HEADER	left:3	top:3	right:3	bottom:3
type:LabelRegionType.HEADER	left:1	top:4	right:4	bottom:4
type:LabelRegionType.DATA	left:1	top:5	right:4	bottom:6
type:LabelRegionType.DATA	left:1	top:7	right:1	bottom:7
type:LabelRegionType.DATA	left:3	top:7	right:4	bottom:7
t

Test Set Validation of fold 0:   3%|▎         | 2/63 [00:56<28:39, 28.19s/it]
0it [00:56, ?it/s]


KeyboardInterrupt: 